# Abstract

Test individual test comparison associations

# Environment

In [1]:
# Setup debugging log
import logging
logger = logging.getLogger('jwst.associations')
handler = logging.StreamHandler()
logger.addHandler(handler)
handler.setLevel(logging.DEBUG)
logger.setLevel(logging.DEBUG)

In [2]:
from copy import copy
import gc
from glob import glob
from os import (
    listdir, 
    mkdir,
    path
)
import pdb
import re
import shutil
from tempfile import TemporaryDirectory

In [3]:
import numpy as np

In [4]:
from jwst.associations import (
    AssociationPool,
    AssociationRegistry,
    generate
)
from jwst.associations.main import (Main, constrain_on_candidates)
from jwst.associations.lib.rules_level3_base import _EMPTY

In [5]:
from jwst.associations.tests.helpers import (
    combine_pools,
    registry_level3_only,
    registry_level2_only,
    t_path,
)

In [6]:
from jwst.associations.tests.test_standards import (
    DEF_ARGS,
    LV2_ONLY_ARGS,
    LV3_ONLY_ARGS,
    MakePars,
    standards
)

In [7]:
# Folders
DATA_DIR = path.join(
    '..', 'data'
)
NEW_ASN_DIR = path.join(
    DATA_DIR,
    'new_asn_standards'
)
SDP_DIR = path.join(
    '..', 'data', 'sdp'
)
SDP_POOLS = path.join(
    SDP_DIR, 'pools'
)
SDP_NEW_ASN_DIR = path.join(
    SDP_DIR, 'new_asns'
)

# Library

In [8]:
def make_standards(pool_root, 
                   main_args=None,
                   source=DATA_DIR,
                   outdir=NEW_ASN_DIR
                  ):
    """Make the association standards for a pool"""
    pool_path = path.join(
        source,
        pool_root + '.csv'
    )
    pool = combine_pools(pool_path)
    if main_args is None:
        main_args = []
    with TemporaryDirectory() as tmp_path:
        args = main_args + ['-v', '-p', tmp_path]
        results = Main(
            args,
            pool=pool
        )
        asn_paths = listdir(tmp_path)
        for asn_path in asn_paths:
            matches = re.match('[^-]+(-.+)', asn_path)
            asn_tail = matches.group(1)
            from_path = path.join(tmp_path, asn_path)
            to_path = path.join(outdir, pool_root + asn_tail)
            shutil.move(from_path, to_path)
        pool.write(path.join(outdir, pool_root + '_std.csv'))

In [9]:
def find_test(pool_root, test_pars):
    """Find a particular test setup
    
    Parameters
    ----------
    pool_root: str
        The pool to find in the test parameters.
        
    test_pars: [MakePars[,...]]
        The list of `MakePars` to search
        
    Returns
    -------
    MakePars
        The `MakePars` corresponding to the desired
        pool root.
        
    Raises
    ------
    KeyError if `pool_root` is not found
    """
    for test_par in test_pars:
        if test_par.pool_root == pool_root:
            return test_par
    raise Keyerror('Pool {} not found in test pars'.format(pool_root))

# Main

## Clear result folder

In [12]:
constrain_all_candidates = constrain_on_candidates(None)
constrain_all_candidates

AttrConstraint({'value': None, 'name': 'asn_candidate', 'is_acid': True, 'sources': ['asn_candidate'], 'evaluate': True, 'force_reprocess': False, 'force_unique': True, 'force_undefined': False, 'invalid_values': (None, '', 'NULL', 'Null', 'null', '--', 'N', 'n', 'F', 'f'), 'only_on_match': False, 'onlyif': <function AttrConstraint.__init__.<locals>.<lambda> at 0x114c5c488>, 'required': True, 'found_values': set()})

In [13]:
# constrain_all_candidates.invalid_values = _EMPTY

In [14]:
# constrain_all_candidates

In [15]:
reg_lv2_all_candidates = registry_level2_only(global_constraints=constrain_all_candidates)
reg_lv3_all_candidates = registry_level3_only(global_constraints=constrain_all_candidates)

## sdp/jw87600 double image3 for o029

Now, the second round, the last exposure of the candidate has gone completely missing.

In [16]:
pool_jw87600 = combine_pools('../data/sdp/pools/jw87600_20171108T042226_pool.csv')

In [17]:
constrain_o029 = constrain_on_candidates(['o029'])

In [18]:
constrain_o029

AttrConstraint({'value': '.+(o029).+', 'name': 'asn_candidate', 'is_acid': True, 'sources': ['asn_candidate'], 'evaluate': True, 'force_reprocess': False, 'force_unique': True, 'force_undefined': False, 'invalid_values': (None, '', 'NULL', 'Null', 'null', '--', 'N', 'n', 'F', 'f'), 'only_on_match': False, 'onlyif': <function AttrConstraint.__init__.<locals>.<lambda> at 0x114c5c598>, 'required': True, 'found_values': set()})

In [19]:
reg_lv3_o029 = registry_level3_only(global_constraints=constrain_o029)

Error occurs with level3 only rules, constrain on all candidates

In [20]:
asns = generate(pool_jw87600, reg_lv3_all_candidates)

In [21]:
len(asns)

6

In [22]:
asns

[{
     "asn_type": "image3",
     "asn_rule": "Asn_Image",
     "version_id": null,
     "code_version": "0.9.6",
     "degraded_status": "No known degraded exposures in association.",
     "program": "87600",
     "constraints": "LV3AttrConstraint({'name': 'program', 'sources': ['program'], 'value': '87600'})\nLV3AttrConstraint({'name': 'instrument', 'sources': ['instrume'], 'value': 'niriss'})\nLV3AttrConstraint({'name': 'opt_elem', 'sources': ['filter'], 'value': 'f380m'})\nLV3AttrConstraint({'name': 'opt_elem2', 'sources': ['pupil'], 'value': 'clearp'})\nLV3AttrConstraint({'name': 'target', 'sources': ['targetid'], 'value': '1'})\nLV3AttrConstraint({'name': 'exp_type', 'sources': ['exp_type'], 'value': 'nis_image'})\nLV3AttrConstraint({'name': 'wfsvisit', 'sources': ['visitype'], 'value': 'prime_targeted_fixed'})\nAttrConstraint({'name': 'asn_candidate', 'sources': ['asn_candidate'], 'value': \"\\\\(\\\\'o029\\\\'\\\\,\\\\ \\\\'observation\\\\'\\\\)\"})",
     "asn_id": "o029",
  

In [23]:
for idx, asn in enumerate(asns):
    if asn['asn_id'] == 'o029' and asn['asn_type'] == 'image3':
        print(idx, asn)

0 jw87600-o029_image3_001_asn with 1 products
Rule=Asn_Image
LV3AttrConstraint({'name': 'program', 'sources': ['program'], 'value': '87600'})
LV3AttrConstraint({'name': 'instrument', 'sources': ['instrume'], 'value': 'niriss'})
LV3AttrConstraint({'name': 'opt_elem', 'sources': ['filter'], 'value': 'f380m'})
LV3AttrConstraint({'name': 'opt_elem2', 'sources': ['pupil'], 'value': 'clearp'})
LV3AttrConstraint({'name': 'target', 'sources': ['targetid'], 'value': '1'})
LV3AttrConstraint({'name': 'exp_type', 'sources': ['exp_type'], 'value': 'nis_image'})
LV3AttrConstraint({'name': 'wfsvisit', 'sources': ['visitype'], 'value': 'prime_targeted_fixed'})
AttrConstraint({'name': 'asn_candidate', 'sources': ['asn_candidate'], 'value': "\\(\\'o029\\'\\,\\ \\'observation\\'\\)"})
Products:
	jw87600-o029_t001_niriss_f380m-clearp with 1 members
5 jw87600-o029_image3_002_asn with 1 products
Rule=Asn_Image
LV3AttrConstraint({'name': 'program', 'sources': ['program'], 'value': '87600'})
LV3AttrConstraint

In [ ]:
asns[19]['products'][0]['members']

In [ ]:
asns[20]['products'][0]['members']

Try on just the one exposure. Works as expected

In [ ]:
pool_jw87600_single = combine_pools('../data/sdp/pools/jw87600_single_pool.csv')

In [ ]:
pool_jw87600_single.show_in_notebook()

In [ ]:
asns = generate(pool_jw87600_single, reg_lv3_all_candidates)

In [ ]:
len(asns)

In [ ]:
asns

Try the whole candidate. Works fine

In [ ]:
pool_jw87600_o029 = combine_pools('../data/sdp/pools/jw87600_o029_pool.csv')

In [ ]:
pool_jw87600_o029.show_in_notebook()

In [ ]:
asns = generate(pool_jw87600_o029, reg_lv3_all_candidates)

In [ ]:
len(asns)

In [ ]:
for asn in asns:
    if asn['asn_id'] == 'o029' and asn['asn_type'] == 'image3':
        print(asn)

Remove the offending exposure, see if something else appears. Nope, its gone and the other doesn't start duplicating.

In [ ]:
pool_jw87600_removed = combine_pools('../data/sdp/pools/jw87600_exp_removed_pool.csv')

In [ ]:
asns = generate(pool_jw87600_removed, reg_lv3_all_candidates)

In [ ]:
for asn in asns:
    if asn['asn_id'] == 'o029' and asn['asn_type'] == 'image3':
        print(asn)

Remove extraneous exposures around o029. Duplication occurs. Has something to do with the intervening ones.

In [ ]:
pool_jw87600_surrounding_removed = combine_pools('../data/sdp/pools/jw87600_surrounding_removed_pool.csv')

In [ ]:
asns = generate(pool_jw87600_surrounding_removed, reg_lv3_all_candidates)

In [ ]:
for asn in asns:
    if asn['asn_id'] == 'o029' and asn['asn_type'] == 'image3':
        print(asn)

Will work here removing intervening exposures until problem goes away.

_And have actually found a single other exposure that causes the issue. Problem pool now contains only two exposures. An exposure without a candidate, and a exposure with a candidate. And the without candidate must be first. The expectation is to get two associations, one discovered that has both the exposures in it, the other the observation candidate with the one exposure. Instead, we get the observation candidate twice._

In [ ]:
pool_jw87600_intervening_removed = combine_pools('../data/sdp/pools/jw87600_intervening_removed_pool.csv')

In [ ]:
asns = generate(pool_jw87600_intervening_removed, reg_lv3_all_candidates)

In [ ]:
for idx, asn in enumerate(asns):
    if asn['asn_id'] == 'o029' and asn['asn_type'] == 'image3':
        print(idx, asn)

In [ ]:
asns

In [ ]:
pool_jw87600_intervening_removed.show_in_notebook()

Now let's debug the hell out of this...

In [ ]:
pdb.run("generate(pool_jw87600_intervening_removed, reg_lv3_all_candidates)")

Now lets run the Main and see what is up.

In [ ]:
cmd_args = [
    '--dry-run',
]

In [ ]:
main_generated = Main(cmd_args, pool=pool_jw87600_intervening_removed)

In [ ]:
main_generated.associations

## pool_005_niriss looking for level2 spectral

In [ ]:
pool_005_niriss = combine_pools('../data/pool_005_spec_niriss.cs')

In [ ]:
pool_005_niriss.show_in_notebook()

In [ ]:
asns = generate(pool_005_niriss, reg_lv2_all_candidates)

In [ ]:
len(asns)

In [ ]:
pdb.run("generate(pool_005_niriss, reg_lv2_all_candidates)")

## pool_004_wfs looking for level2 image

In [ ]:
pool_004_wfs = combine_pools('../data/pool_004_wfs.csv')

In [ ]:
pool_004_wfs.show_in_notebook()

In [ ]:
pdb.run("generate(pool_004_wfs, reg_lv2_all_candidates)")

In [ ]:
asns = generate(pool_004_wfs, reg_lv2_all_candidates)

In [ ]:
len(asns)

In [ ]:
asns

## 82600

In [ ]:
all_candidates = copy(LV3_ONLY_ARGS)
all_candidates.extend(['--all-candidates'])

In [ ]:
pool_82600_fpath = '../data/sdp/pools/jw82600_20171108T041350_pool.csv'
pool_82600 = AssociationPool.read(pool_82600_fpath)
pool_82600_path, pool_82600_root = path.split(pool_82600_fpath)
pool_82600_root = path.splitext(pool_82600_root)[0]

In [ ]:
reg_lv3 = registry_level3_only()
reg_all_candidates = registry_level3_only(global_constraints=constrain_on_candidates(None))

In [ ]:
pdb.run("asns = generate(pool_82600, reg_all_candidates)")

In [ ]:
asns = generate(pool_82600, reg_all_candidates)

In [ ]:
len(asns)

In [ ]:
asns

In [ ]:
gc.collect()

In [ ]:
pool_82600[pool_82600['asn_candidate'] != 'null']['exp_type']

In [ ]:
pool_mega = combine_pools('../data/mega_pool.csv')

In [ ]:
pdb.run("generate(pool_mega, reg_all_candidates)")

In [ ]:
make_standards(
    pool_82600_root,
    source= pool_82600_path,
    outdir=SDP_NEW_ASN_DIR
)

## Coronographic test

In [ ]:
nrc_coron_pars = find_test('pool_013_coron_nircam', standards)

In [ ]:
c1000_args = copy(LV3_ONLY_ARGS)
c1000_args.extend(['-i', 'c1000', 'c1001'])

In [ ]:
make_standards(
    nrc_coron_pars.pool_root,
    main_args=all_candidates,
    source=DATA_DIR,
    outdir=NEW_ASN_DIR
)

In [ ]:
nrc_coron_pool = combine_pools(path.join(DATA_DIR, nrc_coron_pars.pool_root + '.csv'))

In [ ]:
nrc_coron_pool['filename','targetid', 'is_psf', 'asn_candidate']

In [ ]:
reg_coron = copy(reg_lv3)

In [ ]:
names = [name for name in reg_coron]
for name in names:
    if name != 'Asn_Coron':
        del reg_coron[name]

In [ ]:
asns = generate(nrc_coron_pool, reg_coron)

In [ ]:
len(asns)

In [ ]:
asns

In [ ]:
pdb.run("generate(nrc_coron_pool, reg_coron)")

## Make the SPD-based associations

In [ ]:
pools = glob(path.join(SDP_POOLS, '*.csv'))

In [ ]:
for pool in pools:
    if MAKE_ALL:
        pool_dir, pool = path.split(pool)
        pool, ext = path.splitext(pool)
        make_standards(
            pool,
            source=pool_dir,
            outdir=SDP_NEW_ASN_DIR
        )